In [1]:
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Drugs table

In [2]:
drugs = pd.read_excel('bcdrugs.xlsx')
drugs.shape

(1203, 36)

Drugs columns = 'Drug ID', 'Drug Name', 'CAS Number', 'Chemical Name',
       'New Molecular Entity', 'Drug Interactions', 'Comparative Efficacy',
       'Highest Development Phase', 'Indication', 'Molecular Formula',
       'Mechanism Of Action', 'Organisation',
       'Pharmacodynamic Characteristics', 'Route Of Administration',
       'Route Of Elimination', 'Pharmacokinetic Characteristics',
       'Adverse Events', 'Therapeutic Class (EphMRA)',
       'Therapeutic Class (WHO)', 'Synonyms/Brand Names',
       'Antimicrobial Activity', 'Immunogenicity',
       'Last Significant Update Date', 'Therapeutic Area',
       'Orphan Designation', 'License Availability', 'Brands', 'On Fastrack',
       'Fastrack country', 'On PRIME', 'PRIME country', 'Biomarkers-Inclusion',
       'Biomarkers-Exclusion', 'Biomarkers-Outcome Measure',
       'Biomarkers-Safety', 'Biomarkers-not specified'

In [3]:
drugs['Highest Development Phase'].value_counts()

Marketed                                     175
Preclinical                                  167
Phase II                                     149
No development reported (Preclinical)         87
Phase III                                     84
Discontinued (II)                             84
Phase I                                       65
Discontinued (Preclinical)                    60
Discontinued (III)                            46
Phase I/II                                    44
Research                                      35
Discontinued (I)                              25
No development reported (I)                   21
No development reported (Research)            21
No development reported (II)                  20
Registered                                    15
Phase II/III                                  14
Discontinued (I/II)                           11
Preregistration                               10
No development reported (Clinical)            10
Discontinued (Prereg

## Trials table

In [4]:
trials = pd.read_excel('bcdrugsct.xlsx')
trials.shape

(4529, 41)

In [5]:
cols = ['Trial ID','Mechanism of action', 'Drug class (therapeutic effect)', 'Drug class (chemical)', \
        'Indication', 'Organisations', 'Trial Design', 'Location', 'Phase of Trial', 'Subject Age',\
        'Planned Subject Number', 'Trial Centre Details', 'Lead Centre', 'Trial Initiation date', \
        'Trial End date', 'Trial Status', 'Trial History', 'Diseases treated', 'Primary Drugs']
smaller_trials = trials[cols]
date_cols = ['Phase of Trial','Trial Initiation date', 'Trial End date', 'trial_start','trial_end']

In [7]:

def phase_1(drug_name):
    '''
    take earliest Phase I date and latest Phase II
    '''
    tucatinib_trials = trials[trials['Primary Drugs']==drug_name]
    trial_phases = ['Phase I', 'Phase II', 'Phase III']
    cleaned_tucatinib_trials = tucatinib_trials[tucatinib_trials['Phase of Trial'].isin(trial_phases)]
    phase_dummies = pd.get_dummies(cleaned_tucatinib_trials['Phase of Trial'])
    cleaned_tucatinib_trials_dummies = cleaned_tucatinib_trials.join(phase_dummies)
    #cleaned_tucatinib_trials_dummies[['Trial Initiation date', 'Trial End date']]
    df = cleaned_tucatinib_trials.join(phase_dummies)

    #not DRY as the answers to make multiargument apply functions involved more complicated things than I wanted.
    def start_to_datetime(x):
        date_str = re.findall('[\d\s\w]*', x['Trial Initiation date'])[0].strip()
        if date_str == '':
            return False
        else:
            return pd.to_datetime(date_str)

    def end_to_datetime(x):
        date_str = re.findall('[\d\s\w]*', x['Trial End date'])[0].strip()
        if date_str == '':
            return False
        else:
            return pd.to_datetime(date_str)

    df['trial_start'] = df.apply(start_to_datetime, axis=1)
    df['trial_end'] = df.apply(end_to_datetime, axis=1)

    t_phase_1_min = min(list(df[(df['trial_start']!=False) & (df['Phase of Trial']=='Phase I')]['trial_start'].values))
    t_phase_1_max = max(list(df[(df['trial_start']!=False) & (df['Phase of Trial']=='Phase I')]['trial_start'].values))

    return f'min is {t_phase_1_min} and max is {t_phase_1_max}'

phase_1('Xentuzumab')


# df['split_start_date'] = df['Trial Initiation date'].apply(lambda x: x.split(' '))
# df['split_end_date'] = df['Trial End date'].apply(lambda x: x.split(' '))
# df2 = df[df['split_start_date'].apply(lambda x: len(x[:3]))==3]
# df3 = df2[df2['split_end_date'].apply(lambda x: len(x[:3]))==3]
# df3[date_cols]
#df3.apply(lambda x: ' '.join(x['split_start_date']))
# in_progress_trials = trials[trials['split_start_date'].apply(lambda x: len(x[:3]))==3]
# planned_trials = trials[trials['split_start_date'].apply(lambda x: len(x[:3]))==2]
# # one_split = trials['split_end_date'].values[0]
# # pd.to_datetime(' '.join(one_split[:3]))







ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series

In [8]:
phase_two = smaller_trials[smaller_trials['Phase of Trial'] == 'Phase II']
phase_three = smaller_trials[smaller_trials['Phase of Trial'] == 'Phase III']

In [9]:
unique_drugs =list(smaller_trials['Primary Drugs'].unique())

In [10]:
trials[trials['Primary Drugs']=='Tucatinib']

,Trial ID,Trial Title,Mechanism of action,Drug class (therapeutic effect),Drug class (chemical),Drug class (biological),Indication,Organisations,Trial Design,Trial Endpoints,...,Trial End date,Trial Status,Trial History,Diseases treated,Biomarkers-Inclusion,Biomarkers-Exclusion,Biomarkers-Outcome Measure,Biomarkers-Safety,Biomarkers-not specified,Primary Drugs
1,700240183,"A Phase 1b, Open-label Study to Assess the Saf...","Antibody-dependent cell cytotoxicity, Antimeta...","Antineoplastics, Antineoplastics, Antineoplastics","2 ring heterocyclic compounds, Diamines, Furan...",Monoclonal antibodies,"Advanced breast cancer, Male breast cancer","Cascadian Therapeutics, Cascadian Therapeutics...","multicentre, open, parallel, prospective",Incidence of adverse events\ntime_frame: Up to...,...,01 Jul 2019 (planned),"Active, no longer recruiting",2019-03-06|Other trial event|Last checked agai...,Advanced breast cancer|treatment|second-line o...,HER2,HER2,NaN,NaN,NaN,Tucatinib
10,700257487,"Phase 2 Randomized, Double-Blinded, Controlled...","Antibody-dependent cell cytotoxicity, Antibody...","Antineoplastics, Antineoplastics, Antineoplast...","2 ring heterocyclic compounds, Diamines, Furan...","Monoclonal antibodies, Monoclonal antibodies","Advanced breast cancer, Carcinoma, Male breast...","Cascadian Therapeutics, Cascadian Therapeutics...","double-blind, multicentre, parallel, prospecti...",Progression-free survival (PFS) per RECIST 1.1...,...,01 Jul 2021 (planned),Recruiting,2019-03-06|Other trial event|Last checked agai...,Advanced breast cancer|treatment|second-line o...,HER2,HER2,NaN,NaN,NaN,Tucatinib
16,700304216,"A Phase 1, Open-Label, Fixed-Sequence, Drug-Dr...","AMP activated protein kinase stimulants, Emiss...","Antineoplastics, Antihyperglycaemics","2 ring heterocyclic compounds, Diamines, Furan...",NaN,"Breast cancer, Colorectal cancer","Seattle Genetics, Seattle Genetics","open, prospective",Maximum observed plasma concentration (Cmax) o...,...,01 Mar 2019 (planned),Recruiting,2019-03-06|Other trial event|Last checked agai...,Breast cancer|treatment|; Colorectal cancer|tr...,NaN,NaN,NaN,NaN,NaN,Tucatinib
478,700302859,"A Phase 1, Randomized, Partially Double-Blind,...","DNA gyrase inhibitors, ERBB 2 receptor antagon...","Antineoplastics, Antibacterials","2 ring heterocyclic compounds, Diamines, Furan...",NaN,"Breast cancer, Colorectal cancer","Seattle Genetics, Seattle Genetics","crossover, double-blind, prospective, randomised",Placebo-corrected change-from-baseline in QTcF...,...,01 Mar 2019 (planned),Recruiting,2019-02-03|Other trial event|Last checked agai...,Breast cancer|treatment|; Colorectal cancer|tr...,NaN,NaN,NaN,NaN,NaN,Tucatinib
481,700238859,"A Phase 1b, Open-label Study to Assess the Saf...","Apoptosis stimulants, ERBB 2 receptor antagoni...","Antineoplastics, Antineoplastics","2 ring heterocyclic compounds, Diamines, Furan...","Immunoconjugates, Immunotoxins, Monoclonal ant...","Advanced breast cancer, Male breast cancer","Cascadian Therapeutics, Cascadian Therapeutics...","multicentre, open, prospective",Incidence and severity of adverse events\nsafe...,...,01 Jul 2019 (planned),"Active, no longer recruiting",2019-02-01|Other trial event|Last checked agai...,Advanced breast cancer|treatment|stage IV; Mal...,HER2,HER2,NaN,NaN,NaN,Tucatinib
489,700301177,"An Open-label, Non-randomized, Single-dose, Pa...",ERBB 2 receptor antagonists,Antineoplastics,"2 ring heterocyclic compounds, Diamines, Furan...",NaN,"Breast cancer, Colorectal cancer","Seattle Genetics, Seattle Genetics","multicentre, open, parallel, prospective",Maximum observed concentration\ndescription: P...,...,01 May 2019 (planned),Recruiting,2019-02-01|Other trial event|Last checked agai...,Breast cancer|treatment|; Colorectal cancer|tr...,NaN,NaN,NaN,NaN,NaN,Tucatinib
570,700301239,"A Phase 1, Open-Label, Fixed-sequence, 5-part,...","14-alpha demethylase inhibitors, Angiogenesis ...","Antineoplastics, Antihyperlipidaemics, Antifun...","2 ring he

In [11]:
def trial_history_to_table(th):
    rows = str.split(th, ';')
    #columns_trial_1 = [str.split(i, '|') for i in rows_trial_1]
    table = [str.split(row, '|') for row in rows]
    return pd.DataFrame(table,columns=['Date','Status','Notes'])

trial_history_to_table(phase_three['Trial History'].values[0])

,Date,Status,Notes
0,2019-03-06,Other trial event,Last checked against ClinicalTrials.gov record.
1,2019-01-25,Other trial event,Last checked against the European Clinical Tri...
2,2018-12-26,Other trial event,New source identified and integrated (European...
3,2018-12-19,Status change - recruiting,Status changed from not yet recruiting to recr...
4,2018-11-06,New trial record,New trial record
